In [ ]:
%load_ext autoreload
%autoreload 2

## Requirements
* The dataset can be downloaded from [this Kaggle competition](https://www.kaggle.com/c/ieee-fraud-detection).
* In addition to the [Anaconda](https://www.anaconda.com) libraries, you need to install `altair`, `vega_datasets`, `pyod` and `scikit-learn` version 0.24 or higher.
* You also need to set up an AWS account and install `awscli` and `sagemaker-python-sdk`.

In [2]:
import gc
import multiprocessing
import os
import pickle
import numpy as np
import pandas as pd
import altair as alt
from scipy.interpolate import interp1d
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.manifold import TSNE
from sklearn.metrics import (
    average_precision_score,
    precision_recall_curve,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import train_test_split
from pyod.models.copod import COPOD
from pyod.models.iforest import IForest

In [3]:
def dump_pickle(file_path, obj):
    with open(file_path, "wb") as file:
        pickle.dump(obj, file)


def load_pickle(file_path):
    with open(file_path, "rb") as file:
        obj = pickle.load(file)
    return obj


def cast_data_type(series):
    column_type = series.dtype
    if column_type != np.object:
        val_min = series.min()
        val_max = series.max()
        if column_type == np.int_:
            if val_min > np.iinfo(np.int8).min and val_max < np.iinfo(np.int8).max:
                series = series.astype(np.int8)
            elif val_min > np.iinfo(np.int16).min and val_max < np.iinfo(np.int16).max:
                series = series.astype(np.int16)
            elif val_min > np.iinfo(np.int32).min and val_max < np.iinfo(np.int32).max:
                series = series.astype(np.int32)
            elif val_min > np.iinfo(np.int64).min and val_max < np.iinfo(np.int64).max:
                series = series.astype(np.int64)
        else:
            if (
                val_min > np.finfo(np.float16).min
                and val_max < np.finfo(np.float16).max
            ):
                series = series.astype(np.float16)
            elif (
                val_min > np.finfo(np.float32).min
                and val_max < np.finfo(np.float32).max
            ):
                series = series.astype(np.float32)
            else:
                series = series.astype(np.float64)
    return series


def reduce_mem_usage(df):
    mem_usage = df.memory_usage().sum() / 1024 ** 2
    print(f"Memory usage of dataframe is {mem_usage:0.2f} MB.")
    df = df.apply(cast_data_type)
    opt_mem_usage = df.memory_usage().sum() / 1024 ** 2
    print(f"Memory usage after optimization is {opt_mem_usage:0.2f} MB.")
    print(
        f"Decreased by {(100 * (mem_usage - opt_mem_usage) / mem_usage):0.2f}%."
    )
    return df


def str_to_int(x_val):
    return x_val if pd.isnull(x_val) else str(int(x_val))

#### Data Loading from Local Directory
The Kaggle dataset was saved in the local directory `~/Data/ieee-fraud-detection` in advance.

In [4]:
DATA_DIR = "../../data/ieee-fraud-detection"

In [5]:
train_identity = pd.read_csv(os.path.join(DATA_DIR, "train_identity.csv"))
train_transaction = pd.read_csv(os.path.join(DATA_DIR, "train_transaction.csv"))
test_identity = pd.read_csv(os.path.join(DATA_DIR, "test_identity.csv"))
test_transaction = pd.read_csv(os.path.join(DATA_DIR, "test_transaction.csv"))

df_train = pd.merge(train_transaction, train_identity, on="TransactionID", how="left")
df_test = pd.merge(test_transaction, test_identity, on="TransactionID", how="left")
df_test = df_test.rename(columns={f"id-{i:02d}": f"id_{i:02d}" for i in range(1, 39)})

In [6]:
del train_identity, train_transaction, test_identity, test_transaction
_ = gc.collect()

In [7]:
print(f"Train dataset has {df_train.shape[0]} rows and {df_train.shape[1]} columns.")
print(f"Test dataset has {df_test.shape[0]} rows and {df_test.shape[1]} columns.")

Train dataset has 590540 rows and 434 columns.
Test dataset has 506691 rows and 433 columns.


In [8]:
print(f"The fraud rate is {df_train['isFraud'].mean():.2%}.")

The fraud rate is 3.50%.


In [9]:
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

Memory usage of dataframe is 1959.88 MB.
Memory usage after optimization is 650.48 MB.
Decreased by 66.81%.
Memory usage of dataframe is 1677.73 MB.
Memory usage after optimization is 565.37 MB.
Decreased by 66.30%.


# Exploratory Data Analysis

In [10]:
prop_of_missing_values = (
    df_train[df_train.columns.difference(["TransactionID", "isFraud"])].isnull().sum()
    / df_train.shape[0]
).reset_index()
prop_of_missing_values.columns = ["feature", "prop_of_missing_values"]
source = prop_of_missing_values.sample(100, random_state=42)

highlight = alt.selection(
    type="single", on="mouseover", fields=["feature"], nearest=True
)
bars = (
    alt.Chart(source)
    .mark_bar()
    .encode(
        x=alt.X("feature:N", axis=alt.Axis(title="Feature"), sort="-y"),
        y=alt.Y(
            "prop_of_missing_values:Q", axis=alt.Axis(title="Percentage", format=".0%")
        ),
        color=alt.Color("prop_of_missing_values:Q", legend=None),
        opacity=alt.condition(~highlight, alt.value(1.0), alt.value(0.5)),
        tooltip=["feature:N", alt.Tooltip("prop_of_missing_values:Q", format=".2%")],
    )
    .add_selection(highlight)
)
bars.properties(
    title="Proportions of Missing Values", width=1200, height=200
).configure_axisX(labelAngle=-45, labelFontSize=8)

alt.Chart(...)

In [11]:
cat_features = pd.Index(
    [
        "ProductCD",
        "addr1",
        "addr2",
        "P_emaildomain",
        "R_emaildomain",
        "DeviceType",
        "DeviceInfo",
    ]
    + [f"card{i}" for i in range(1, 7)]
    + [f"M{i}" for i in range(1, 10)]
    + [f"id_{i}" for i in range(12, 39)]
)
num_features = df_train.columns.difference(
    pd.Index(["TransactionID", "TransactionDT", "isFraud"]) | cat_features
)
all_features = cat_features | num_features

In [12]:
print(
    f"There are {len(cat_features)} categorical features and {len(num_features)} numeric features."
)

There are 49 categorical features and 382 numeric features.


In [13]:
int_cat_features = df_train[cat_features].select_dtypes("number").columns
df_train[int_cat_features] = df_train[int_cat_features].applymap(str_to_int)

int_cat_features = df_test[cat_features].select_dtypes("number").columns
df_test[int_cat_features] = df_test[int_cat_features].applymap(str_to_int)

In [14]:
source = df_train[cat_features].nunique().reset_index()
source.columns = ["feature", "cardinality"]

bars = alt.Chart(source).mark_bar().encode(
    x=alt.X("feature:N", axis=alt.Axis(title="Feature"), sort="-y"),
    y=alt.Y("cardinality:Q", axis=alt.Axis(title="Count")),
    tooltip=["feature:N", "cardinality:Q"]
)
bars.properties(title="Cardinalities of Categorical Features", width=1000, height=200).configure_axisX(labelAngle=-45)

alt.Chart(...)

In [15]:
def plot_histogram(values, index, bins=20, bar_size=15, height=200, n_digits=0):
    hist, bin_edges = np.histogram(values, bins=bins)
    source = (
        pd.DataFrame(hist, index=bin_edges[:-1], columns=["count"])
        .reset_index()
        .rename(columns={index: "count", "index": index})
    )
    stats = pd.DataFrame(
        {
            "mean": [round(values.mean(), 2)],
            "median": [round(np.quantile(values, 0.5), 2)],
        }
    )
    eps = 0.02

    bars = (
        alt.Chart(source)
        .transform_joinaggregate(total_count="sum(count)")
        .transform_calculate(pecent_of_total="datum.count / datum.total_count")
        .mark_bar(size=bar_size)
        .encode(
            x=alt.X(
                f"{index}:Q",
                axis=alt.Axis(title=None, format=f".{n_digits}f"),
                scale=alt.Scale(
                    domain=[bin_edges[0] - eps, bin_edges[-1] + eps], nice=False
                ),
            ),
            y=alt.Y("pecent_of_total:Q", axis=alt.Axis(title=None, format=".0%")),
            tooltip=[
                alt.Tooltip(f"{index}:Q", format=f".{n_digits}f"),
                alt.Tooltip("count:Q", format=".0f"),
            ],
        )
    )
    rule_mean = (
        alt.Chart(stats)
        .mark_rule(color="#ff7f0e", size=1.5, strokeDash=[3, 2])
        .encode(x="mean:Q", tooltip=["mean", "mean:Q"])
    )
    rule_median = (
        alt.Chart(stats)
        .mark_rule(color="#2ca02c", size=1.5, strokeDash=[3, 2])
        .encode(x="median:Q", tooltip=["median", "median:Q"])
    )
    return (bars + rule_mean + rule_median).properties(
        title=f"Histogram of '{index}'", height=height
    )

In [16]:
np.random.seed(42)
missing_value_ratios = df_train[num_features].isnull().sum() / df_train.shape[0]
value_counts = df_train[num_features].nunique()
selected_features = num_features[(missing_value_ratios < 0.5) & (value_counts > 99)]
selected_features = np.random.permutation(selected_features)[:20]

In [17]:
charts = []
for feature in selected_features:
    charts.append(
        plot_histogram(
            df_train[feature].dropna(),
            feature,
            bins=20,
            bar_size=12.5,
            height=100,
            n_digits=0,
        )
    )

rows = []
for i, chart in enumerate(charts):
    if (i % 3 == 2) or (i == len(charts) - 1):
        rows.append(alt.HConcatChart(hconcat=charts[i - (i % 3) : i + 1]))
alt.VConcatChart(vconcat=rows[:3]).configure_axisY(
    labelAlign="left", labelLimit=30, labelPadding=30
).configure_axisX(labelAngle=-45)

alt.VConcatChart(...)

In [18]:
corr_matrix = df_train[selected_features].corr()
source = corr_matrix.stack().reset_index()
source.columns = ["feature_x", "feature_y", "correlation"]

base = alt.Chart(source).encode(
    x=alt.X("feature_x:N", axis=alt.Axis(ticks=False, title="Feature")),
    y=alt.Y("feature_y:N", axis=alt.Axis(ticks=False, title="Feature")),
)
text = base.mark_text(size=10).encode(
    text=alt.Text("correlation", format=".0%"),
    color=alt.condition(
        alt.datum.correlation > 0.5, alt.value("white"), alt.value("black")
    ),
)
heatmap = base.mark_rect().encode(
    color=alt.Color(
        "correlation:Q", legend=alt.Legend(title="Correlation", titleFontSize=9)
    )
)
(heatmap + text).properties(
    title="Correlation Matrix", width=500, height=500
).configure_axisX(labelAngle=-45)

alt.LayerChart(...)

# Data Splitting and Preprocessing

In [19]:
df_train[cat_features] = df_train[cat_features].astype("str")
df_test[cat_features] = df_test[cat_features].astype("str")

df_X_train, df_X_valid, df_y_train, df_y_valid = train_test_split(
    df_train[all_features],
    df_train["isFraud"],
    test_size=0.2,
    random_state=42,
    stratify=df_train["isFraud"],
)

In [20]:
n_jobs = int(0.75 * multiprocessing.cpu_count())
cat_pipeline = make_pipeline(
    OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan),
    SimpleImputer(strategy="constant", fill_value=-1),
)
num_pipeline = make_pipeline(StandardScaler(), SimpleImputer(strategy="median"))
transformer = make_column_transformer(
    (cat_pipeline, cat_features), (num_pipeline, num_features)
)

X_train = transformer.fit_transform(df_X_train)
X_valid = transformer.transform(df_X_valid)

### Data Visualization with t-SNE

In [21]:
tsne = TSNE(
    n_components=2,
    perplexity=50.0,
    early_exaggeration=12.0,
    learning_rate=200.0,
    random_state=42,
    n_jobs=n_jobs,
)
manifold = tsne.fit_transform(X_valid[:5000])

In [22]:
source = pd.DataFrame(
    np.c_[manifold, df_y_valid.iloc[:5000].values],
    columns=["feature_x", "feature_y", "is_fraud"],
)
source["is_fraud"] = source["is_fraud"].map(lambda x: str(int(x)))

brush = alt.selection(type="interval")
base = alt.Chart(source).add_selection(brush)
points = base.mark_point(size=5).encode(
    x=alt.X("feature_x", title=None),
    y=alt.Y("feature_y", title=None),
    color=alt.condition(
        brush,
        alt.Color("is_fraud:N", legend=alt.Legend(title="Fraudulent", titleFontSize=9)),
        alt.value("grey"),
    ),
)
tick_axis = alt.Axis(labels=False, domain=False, ticks=False)
x_ticks = base.mark_tick().encode(
    alt.X("feature_x", title="Feature", axis=tick_axis),
    alt.Y("is_fraud", title=None, axis=tick_axis),
    color=alt.condition(brush, "is_fraud", alt.value("lightgrey")),
)
y_ticks = base.mark_tick().encode(
    alt.X("is_fraud", title=None, axis=tick_axis),
    alt.Y("feature_y", title="Feature", axis=tick_axis),
    color=alt.condition(brush, "is_fraud", alt.value("lightgrey")),
)
(y_ticks | (points & x_ticks)).properties(
    title="Scatter Plot of Manifold with t-SNE"
).configure_title(anchor="middle")

alt.HConcatChart(...)

# Model Training and Prediction
## Fitting and Prediction with PyOD Isolation Forest and COPOD 

In [23]:
%%time
if_estimator = IForest(
    n_estimators=100, behaviour="new", n_jobs=n_jobs, random_state=42
)
_ = if_estimator.fit(X_train)
if_scores = if_estimator.predict_proba(X_valid)

CPU times: user 9min 28s, sys: 3min 26s, total: 12min 55s
Wall time: 7min 39s


In [24]:
%%time
cop_estimator = COPOD() 
_ = cop_estimator.fit(X_train)
cop_scores = cop_estimator.predict_proba(X_valid)

CPU times: user 4min 5s, sys: 2min 3s, total: 6min 9s
Wall time: 5min 52s


## Fitting with Amazon SageMaker Random Cut Forest

In [25]:
import boto3
import botocore
import sagemaker
from sagemaker import RandomCutForest

In [26]:
def check_bucket_permission(bucket):
    permission = False
    try:
        boto3.Session().client("s3").head_bucket(Bucket=bucket)
    except botocore.exceptions.ParamValidationError:
        print(
            "Hey! You either forgot to specify your S3 bucket \
            or you gave your bucket an invalid name!"
        )
    except botocore.exceptions.ClientError as error:
        if error.response["Error"]["Code"] == "403":
            print(f"Hey! You don't have permission to access the bucket, {bucket}.")
        elif error.response["Error"]["Code"] == "404":
            print(f"Hey! Your bucket, {bucket}, doesn't exist!")
        else:
            raise
    else:
        permission = True
    return permission

In [27]:
sagemaker_session = sagemaker.Session()
BUCKET = sagemaker_session.default_bucket()
PREFIX = "ieee-fraud-detection"
region = boto3.Session().region_name
role = sagemaker.get_execution_role()

if check_bucket_permission(BUCKET):
    print(f"Input/output will be stored in: s3://{'_'.join(BUCKET.split('-')[:-1])}_.../{PREFIX}")

Input/output will be stored in: s3://sagemaker_us_east_1_.../ieee-fraud-detection


In [28]:
rcf_estimator = RandomCutForest(
    role=role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    data_location=f"s3://{BUCKET}/{PREFIX}/train/",
    output_path=f"s3://{BUCKET}/{PREFIX}/models",
    num_samples_per_tree=512,
    num_trees=100,
)
_ = rcf_estimator.fit(rcf_estimator.record_set(X_train))

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-06-05 04:57:30 Starting - Starting the training job...
2021-06-05 04:57:33 Starting - Launching requested ML instancesProfilerReport-1622869048: InProgress
......
2021-06-05 04:58:42 Starting - Preparing the instances for training.........
2021-06-05 05:00:43 Downloading - Downloading input data...
2021-06-05 05:01:27 Training - Downloading the training image...
2021-06-05 05:01:54 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[06/05/2021 05:02:00 INFO 140228126345024] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-conf.json: {'num_samples_per_tree': 256, 'num_trees': 100, 'force_dense': 'true', 'eval_metrics': ['accuracy', 'precision_recall_fscore'], 'epochs': 1, 'mini_batch_size': 1000, '_log_level': 'info', '_kvstore': 'dist_async', '_num_kv_servers': 'auto', '_num_gpus': 'auto', '_tuning_objective_metric

#### Uploading Validation Set to S3 Bucket

In [29]:
np.savetxt(os.path.join(DATA_DIR, "X_valid.csv"), X_valid, delimiter=",", fmt="%i")
valid_data_uri = sagemaker_session.upload_data(
    os.path.join(DATA_DIR, "X_valid.csv"), bucket=BUCKET, key_prefix=f"{PREFIX}/valid"
)

## Defining Transformer and Prediction

In [30]:
%%time
rcf_transformer = rcf_estimator.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path=f"s3://{BUCKET}/{PREFIX}/pred",
)
_ = rcf_transformer.transform(
    data=valid_data_uri, content_type="text/csv", split_type="Line"
)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


...............................Docker entrypoint called with argument(s): serve
Running default environment configuration script
[06/05/2021 05:10:55 INFO 140131932772160] loaded entry point class algorithm.serve.server_config:config_api
[06/05/2021 05:10:55 INFO 140131932772160] loading entry points
[06/05/2021 05:10:55 INFO 140131932772160] Loaded iterator creator application/x-recordio-protobuf for content type ('application/x-recordio-protobuf', '1.0')
[06/05/2021 05:10:55 INFO 140131932772160] loaded request iterator application/json
[06/05/2021 05:10:55 INFO 140131932772160] loaded request iterator application/jsonlines
[06/05/2021 05:10:55 INFO 140131932772160] loaded request iterator application/x-recordio-protobuf
[06/05/2021 05:10:55 INFO 140131932772160] loaded request iterator text/csv
[06/05/2021 05:10:55 INFO 140131932772160] loaded response encoder application/json
[06/05/2021 05:10:55 INFO 140131932772160] loaded response encoder application/jsonlines
[06/05/2021 05:10:

#### Downloading Prediction Scores to Local Directory

In [31]:
boto3.resource("s3").meta.client.download_file(
    BUCKET, f"{PREFIX}/pred/X_valid.csv.out", os.path.join(DATA_DIR, "X_valid.csv.out")
)
rcf_scores = (
    pd.read_csv(os.path.join(DATA_DIR, "X_valid.csv.out"), header=None)[0]
    .map(lambda x: eval(x)["score"])
    .values
)

# Model Evaluation

In [32]:
charts = []
for model, score in zip(
    ["Isolation Forest", "COPOD", "Random Cut Forest"],
    [if_scores[:, 1], cop_scores[:, 1], rcf_scores],
):
    charts.append(
        plot_histogram(score, model, bins=50, bar_size=15, height=150, n_digits=2)
    )

alt.VConcatChart(vconcat=charts).configure_axisX(labelAngle=-45)

alt.VConcatChart(...)

In [33]:
roc_curves = [
    x
    for scores in [if_scores[:, 1], cop_scores[:, 1], rcf_scores]
    for x in roc_curve(df_y_valid, scores)
]
aurocs = [
    roc_auc_score(df_y_valid, scores)
    for scores in [if_scores[:, 1], cop_scores[:, 1], rcf_scores]
]

pr_curves = [
    x
    for scores in [if_scores[:, 1], cop_scores[:, 1], rcf_scores]
    for x in precision_recall_curve(df_y_valid, scores)
]
auprcs = [
    average_precision_score(df_y_valid, scores)
    for scores in [if_scores[:, 1], cop_scores[:, 1], rcf_scores]
]

In [34]:
x = np.linspace(0, 1, int(5000 / 3))
source = np.c_[
    x,
    interp1d(roc_curves[0], roc_curves[1])(x),
    interp1d(roc_curves[3], roc_curves[4])(x),
    interp1d(roc_curves[6], roc_curves[7])(x),
]
columns = [
    f"IF (AUROC:{aurocs[0]:0.2%})",
    f"COPOD (AUROC:{aurocs[1]:0.2%})",
    f"RCF (AUROC:{aurocs[2]:0.2%})",
]
source = pd.DataFrame(source, columns=["x"] + columns)
source = pd.melt(source, id_vars=["x"], value_vars=columns)

highlight = alt.selection(
    type="single", on="mouseover", fields=["variable"], nearest=True
)
base = alt.Chart(source).encode(
    x=alt.X("x:Q", title="False Positive Rate"),
    y=alt.Y("value:Q", title="True Positive Rate"),
    color=alt.Color(
        "variable:N", legend=alt.Legend(title="Estimator", orient="bottom-right")
    ),
)
points = (
    base.mark_circle()
    .encode(opacity=alt.value(0))
    .add_selection(highlight)
    .properties()
)
line = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(3))
)
(points + line).properties(title="Receiver Operating Characteristic Curves")

alt.LayerChart(...)

In [35]:
source = np.c_[
    x,
    interp1d(pr_curves[1], pr_curves[0])(x),
    interp1d(pr_curves[4], pr_curves[3])(x),
    interp1d(pr_curves[7], pr_curves[6])(x),
]
columns = [
    f"IF (AUPRC:{auprcs[0]:0.2%})",
    f"COPOD (AUPRC:{auprcs[1]:0.2%})",
    f"RCF (AUPRC:{auprcs[2]:0.2%})",
]
source = pd.DataFrame(source, columns=["x"] + columns)
source = pd.melt(source, id_vars=["x"], value_vars=columns)

highlight = alt.selection(
    type="single", on="mouseover", fields=["variable"], nearest=True
)
base = alt.Chart(source).encode(
    x=alt.X("x:Q", title="Recall"),
    y=alt.Y("value:Q", title="Precision", scale=alt.Scale(domain=[0.0, 1.0])),
    color=alt.Color(
        "variable:N", legend=alt.Legend(title="Estimator", orient="top-right")
    ),
)
points = (
    base.mark_circle()
    .encode(opacity=alt.value(0))
    .add_selection(highlight)
    .properties()
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(3))
)
(points + lines).properties(title="Precision - Recall Curves")

alt.LayerChart(...)

# Model Re-training
## Data Preprocessing, Fitting and Prediction

In [36]:
X_train = transformer.fit_transform(df_train[all_features])
X_test = transformer.transform(df_test[all_features])

In [37]:
%%time
_ = cop_estimator.fit(X_train)
cop_scores = cop_estimator.predict_proba(X_test)

CPU times: user 5min 54s, sys: 2min 41s, total: 8min 35s
Wall time: 8min 21s


In [38]:
submission = pd.DataFrame(
    {"TransactionID": df_test["TransactionID"].values, "isFraud": cop_scores[:, 1]}
)
submission.to_csv("./submission.csv", index=False)